In [82]:
#Of the players who played at their estimated peak age in 2022, which will exceed their 2022 WAR metric?

In [83]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [84]:
#all players 1980-2022
sh = pd.read_csv('ss_all_1980-2022.csv')
sh.head(5)

,Rk,Player,WAR,Season,Age,Team,Lg,G,PA,AB,...,WAR.1,WAA,oWAR,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,Pos
0,1,Barry Bonds,11.9,2001,36,SFG,NL,153,664,476,...,11.9,9.9,12.4,-1.2,116,1,1,1,-5,*7/DH
1,2,Barry Bonds,11.8,2002,37,SFG,NL,143,612,403,...,11.8,9.9,11.8,-0.6,109,0,-2,-2,0,*7/DH
2,3,Cal Ripken Jr.,11.5,1991,30,BAL,AL,162,717,650,...,11.5,9.2,9.2,3.5,56,-1,1,0,23,*6
3,4,Mookie Betts,10.7,2018,25,BOS,AL,136,614,520,...,10.7,8.7,8.7,1.6,65,0,6,7,18,*98/HD4
4,5,Barry Bonds,10.6,2004,39,SFG,NL,147,617,373,...,10.6,8.8,11.5,-1.5,106,1,0,1,-8,*7/HD


In [85]:
fg1980 = pd.read_csv('1980fgbat.csv')
fg1981 = pd.read_csv('1981fgbat.csv')
fg1982 = pd.read_csv('1982fgbat.csv')
fg1983 = pd.read_csv('1983fgbat.csv')
fg1984 = pd.read_csv('1984fgbat.csv')
fg1985 = pd.read_csv('1985fgbat.csv')
fg1986 = pd.read_csv('1986fgbat.csv')
fg1987 = pd.read_csv('1987fgbat.csv')
fg1988 = pd.read_csv('1988fgbat.csv')
fg1989 = pd.read_csv('1989fgbat.csv')
fg1990 = pd.read_csv('1990fgbat.csv')
fg1991 = pd.read_csv('1991fgbat.csv')
fg1992 = pd.read_csv('1992fgbat.csv')
fg1993 = pd.read_csv('1993fgbat.csv')
fg1994 = pd.read_csv('1994fgbat.csv')
fg1995 = pd.read_csv('1995fgbat.csv')
fg1996 = pd.read_csv('1996fgbat.csv')
fg1997 = pd.read_csv('1997fgbat.csv')
fg1998 = pd.read_csv('1998fgbat.csv')
fg1999 = pd.read_csv('1999fgbat.csv')
fg2000 = pd.read_csv('2000fgbat.csv')
fg2001 = pd.read_csv('2001fgbat.csv')
fg2002 = pd.read_csv('2002fgbat.csv')
fg2003 = pd.read_csv('2003fgbat.csv')
fg2004 = pd.read_csv('2004fgbat.csv')
fg2005 = pd.read_csv('2005fgbat.csv')
fg2006 = pd.read_csv('2006fgbat.csv')
fg2007 = pd.read_csv('2007fgbat.csv')
fg2008 = pd.read_csv('2008fgbat.csv')
fg2009 = pd.read_csv('2009fgbat.csv')
fg2010 = pd.read_csv('2010fgbat.csv')
fg2011 = pd.read_csv('2011fgbat.csv')
fg2012 = pd.read_csv('2012fgbat.csv')
fg2013 = pd.read_csv('2013fgbat.csv')
fg2014 = pd.read_csv('2014fgbat.csv')
fg2015 = pd.read_csv('2015fgbat.csv')
fg2016 = pd.read_csv('2016fgbat.csv')
fg2017 = pd.read_csv('2017fgbat.csv')
fg2018 = pd.read_csv('2018fgbat.csv')
fg2019 = pd.read_csv('2019fgbat.csv')
fg2020 = pd.read_csv('2020fgbat.csv')
fg2021 = pd.read_csv('2021fgbat.csv')
fg2022 = pd.read_csv('2022fgbat.csv')

In [86]:
fg1980['Season'] = 1980
fg1981['Season'] = 1981
fg1982['Season'] = 1982
fg1983['Season'] = 1983
fg1984['Season'] = 1984
fg1985['Season'] = 1985
fg1986['Season'] = 1986
fg1987['Season'] = 1987
fg1988['Season'] = 1988
fg1989['Season'] = 1989
fg1990['Season'] = 1990
fg1991['Season'] = 1991
fg1992['Season'] = 1992
fg1993['Season'] = 1993
fg1994['Season'] = 1994
fg1995['Season'] = 1995
fg1996['Season'] = 1996
fg1997['Season'] = 1997
fg1998['Season'] = 1998
fg1999['Season'] = 1999
fg2000['Season'] = 2000
fg2001['Season'] = 2001
fg2002['Season'] = 2002
fg2003['Season'] = 2003
fg2004['Season'] = 2004
fg2005['Season'] = 2005
fg2006['Season'] = 2006
fg2007['Season'] = 2007
fg2008['Season'] = 2008
fg2009['Season'] = 2009
fg2010['Season'] = 2010
fg2011['Season'] = 2011
fg2012['Season'] = 2012
fg2013['Season'] = 2013
fg2014['Season'] = 2014
fg2015['Season'] = 2015
fg2016['Season'] = 2016
fg2017['Season'] = 2017
fg2018['Season'] = 2018
fg2019['Season'] = 2019
fg2020['Season'] = 2020
fg2021['Season'] = 2021
fg2022['Season'] = 2022

In [87]:
#put all of the dfs together into a single dataframe
fg = pd.concat([fg1980,fg1981,fg1982,fg1983,fg1984,fg1985,fg1986,fg1987,fg1988,fg1989,fg1990,fg1991,fg1992,fg1993,fg1994,fg1995,fg1996,fg1997,fg1998,fg1999,fg2000,fg2001,fg2002,fg2003,fg2004,fg2005,fg2006,fg2007,fg2008,fg2009,fg2010,fg2011,fg2012,fg2013,fg2014,fg2015,fg2016,fg2017,fg2018,fg2019,fg2020,fg2021,fg2022])


In [88]:
fg

,Name,Team,G,PA,HR,R,RBI,SB,BB%,K%,...,SLG.2,wOBA.2,xwOBA.2,wRC+.2,BsR.2,Off.2,Def.2,WAR.2,playerid,Season
0,Scott Munninghoff,PHI,4,1,0,1,0,0,0.0%,0.0%,...,3.000,1.649,NaN,1012.0,0.0,1.0,0.1,0.1,1009324,1980
1,Don McCormack,PHI,2,1,0,0,0,0,0.0%,0.0%,...,1.000,0.909,NaN,504.0,0.0,0.5,0.0,0.1,1008412,1980
2,Donnie Moore,STL,11,6,0,1,2,0,0.0%,0.0%,...,1.000,0.779,NaN,414.0,0.0,2.1,0.5,0.3,1009102,1980
3,Brad Gulden,NYY,2,3,1,1,2,0,0.0%,0.0%,...,1.333,0.712,NaN,355.0,0.0,0.9,0.0,0.1,1005133,1980
4,Mark Lee,PIT,4,1,0,0,0,0,100.0%,0.0%,...,0.000,0.708,NaN,363.0,0.0,0.3,0.1,0.0,1007459,1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,Ken Waldichuk,OAK,7,0,0,0,0,0,0.0%,0.0%,...,0.000,0.000,NaN,NaN,0.0,0.0,0.0,0.0,27681,2022
1491,Louie Varland,MIN,5,0,0,0,0,0,0.0%,0.0%,...,0.000,0.000,NaN,NaN,0.0,0.0,0.0,0.0,27691,2022
1492,Bryce Elder,ATL,10,0,0,0,0,0,0.0%,0.0%,...,0.000,0.000,NaN,NaN,0.0,0.0,0.0,0.0,27779,2022
1493,Chase Silseth,LAA,7,0,0,0,0,0,0.0%,0.0%,...,0.000,0.000,NaN,NaN,0.0,0.0,0.0,0.0,30074,2022


In [89]:
#create PlayerTeamSeason in fg df
fg['PlayerTeamSeason'] = fg['Name'].str.replace(" ", "_") + '_' + fg['Team'] + '_' + fg['Season'].map(str)

In [90]:
#use for dupes
print(fg.filter(like='.1').columns)

Index(['G.1', 'PA.1', 'HR.1', 'R.1', 'RBI.1', 'SB.1', 'AVG.1', 'BB%.1', 'K%.1',
       'OBP.1', 'SLG.1', 'ISO.1', 'BABIP.1', 'wOBA.1', 'WAR.1', 'wRC+.1',
       'FB%.1', 'BsR.1', 'Def.1', 'Off.1', 'xwOBA.1'],
      dtype='object')


In [91]:
#drop dupes
fg = fg.drop(['G.1', 'PA.1', 'HR.1', 'R.1', 'RBI.1', 'SB.1', 'AVG.1', 'BB%.1', 'K%.1',
       'OBP.1', 'SLG.1', 'ISO.1', 'BABIP.1', 'wOBA.1', 'WAR.1', 'wRC+.1',
       'FB%.1', 'BsR.1', 'Def.1', 'Off.1', 'xwOBA.1'], axis=1)

In [92]:
#use for dupes
print(fg.filter(like='.2').columns)

Index(['G.2', 'PA.2', 'HR.2', 'R.2', 'RBI.2', 'SB.2', 'BB%.2', 'K%.2', 'ISO.2',
       'BABIP.2', 'AVG.2', 'OBP.2', 'SLG.2', 'wOBA.2', 'xwOBA.2', 'wRC+.2',
       'BsR.2', 'Off.2', 'Def.2', 'WAR.2'],
      dtype='object')


In [93]:
#drop dupes
fg = fg.drop(['G.2', 'PA.2', 'HR.2', 'R.2', 'RBI.2', 'SB.2', 'BB%.2', 'K%.2', 'ISO.2',
       'BABIP.2', 'AVG.2', 'OBP.2', 'SLG.2', 'wOBA.2', 'xwOBA.2', 'wRC+.2',
       'BsR.2', 'Off.2', 'Def.2', 'WAR.2'], axis=1)

In [94]:
#rename FG columns
fg = fg.rename(columns={"1B":"FG_1B","2B":"FG_2B","3B":"FG_3B","AB":"FG_AB","Age":"FG_Age","Age Rng":"FG_Age Rng","AVG":"FG_AVG","AVG+":"FG_AVG+","BABIP":"FG_BABIP","BABIP+":"FG_BABIP+","Balls":"FG_Balls","Barrel%":"FG_Barrel%","Barrels":"FG_Barrels","Bat":"FG_Bat","BB":"FG_BB","BB/K":"FG_BB/K","BB%":"FG_BB%","BB%+":"FG_BB%+","BsR":"FG_BsR","BU":"FG_BU","BUH":"FG_BUH","BUH%":"FG_BUH%","CB%":"FG_CB%","CBv":"FG_CBv","Cent%":"FG_Cent%","Cent%+":"FG_Cent%+","CH-X (pi)":"FG_CH-X (pi)","CH-X (sc)":"FG_CH-X (sc)","CH-Z (pi)":"FG_CH-Z (pi)","CH-Z (sc)":"FG_CH-Z (sc)","CH%":"FG_CH%","CH% (pi)":"FG_CH% (pi)","CH% (sc)":"FG_CH% (sc)","CHv":"FG_CHv","Clutch":"FG_Clutch","Column":"FG_Column","Contact%":"FG_Contact%","Contact% (pi)":"FG_Contact% (pi)","Contact% (sc)":"FG_Contact% (sc)","CS":"FG_CS","CS-X (pi)":"FG_CS-X (pi)","CS-Z (pi)":"FG_CS-Z (pi)","CS% (pi)":"FG_CS% (pi)","CStr%":"FG_CStr%","CSW%":"FG_CSW%","CT%":"FG_CT%","CTv":"FG_CTv","CU-X (pi)":"FG_CU-X (pi)","CU-X (sc)":"FG_CU-X (sc)","CU-Z (pi)":"FG_CU-Z (pi)","CU-Z (sc)":"FG_CU-Z (sc)","CU% (pi)":"FG_CU% (pi)","CU% (sc)":"FG_CU% (sc)","Def":"FG_Def","Dol":"FG_Dol","EP-X (sc)":"FG_EP-X (sc)","EP-Z (sc)":"FG_EP-Z (sc)","EP% (sc)":"FG_EP% (sc)","EV":"FG_EV","Events":"FG_Events","F-Strike%":"FG_F-Strike%","FA-X (pi)":"FG_FA-X (pi)","FA-X (sc)":"FG_FA-X (sc)","FA-Z (pi)":"FG_FA-Z (pi)","FA-Z (sc)":"FG_FA-Z (sc)","FA% (pi)":"FG_FA% (pi)","FA% (sc)":"FG_FA% (sc)","FB":"FG_FB","FB%":"FG_FB%","FB%.1":"FG_FB%.1","FB%+":"FG_FB%+","FBv":"FG_FBv","FC-X (pi)":"FG_FC-X (pi)","FC-X (sc)":"FG_FC-X (sc)","FC-Z (pi)":"FG_FC-Z (pi)","FC-Z (sc)":"FG_FC-Z (sc)","FC% (pi)":"FG_FC% (pi)","FC% (sc)":"FG_FC% (sc)","Fld":"FG_Fld","FO-X (sc)":"FG_FO-X (sc)","FO-Z (sc)":"FG_FO-Z (sc)","FO% (sc)":"FG_FO% (sc)","FRM":"FG_FRM","FS-X (pi)":"FG_FS-X (pi)","FS-X (sc)":"FG_FS-X (sc)","FS-Z (pi)":"FG_FS-Z (pi)","FS-Z (sc)":"FG_FS-Z (sc)","FS% (pi)":"FG_FS% (pi)","FS% (sc)":"FG_FS% (sc)","FT-X (sc)":"FG_FT-X (sc)","FT-Z (sc)":"FG_FT-Z (sc)","FT% (sc)":"FG_FT% (sc)","G":"FG_G","GB":"FG_GB","GB/FB":"FG_GB/FB","GB%":"FG_GB%","GB%+":"FG_GB%+","GDP":"FG_GDP","H":"FG_H","Hard%":"FG_Hard%","Hard%+":"FG_Hard%+","HardHit":"FG_HardHit","HardHit%":"FG_HardHit%","HBP":"FG_HBP","HR":"FG_HR","HR/FB":"FG_HR/FB","HR/FB%+":"FG_HR/FB%+","IBB":"FG_IBB","IFFB":"FG_IFFB","IFFB%":"FG_IFFB%","IFH":"FG_IFH","IFH%":"FG_IFH%","ISO":"FG_ISO","ISO+":"FG_ISO+","K%":"FG_K%","K%+":"FG_K%+","KC-X (sc)":"FG_KC-X (sc)","KC-Z (sc)":"FG_KC-Z (sc)","KC% (sc)":"FG_KC% (sc)","KN-X (pi)":"FG_KN-X (pi)","KN-X (sc)":"FG_KN-X (sc)","KN-Z (pi)":"FG_KN-Z (pi)","KN-Z (sc)":"FG_KN-Z (sc)","KN%":"FG_KN%","KN% (pi)":"FG_KN% (pi)","KN% (sc)":"FG_KN% (sc)","KNv":"FG_KNv","L-WAR":"FG_L-WAR","LA":"FG_LA","LD":"FG_LD","LD%":"FG_LD%","LD+%":"FG_LD+%","Lg":"FG_Lg","maxEV":"FG_maxEV","Med%":"FG_Med%","Med%+":"FG_Med%+","Name":"FG_Name","O-Contact%":"FG_O-Contact%","O-Contact% (pi)":"FG_O-Contact% (pi)","O-Contact% (sc)":"FG_O-Contact% (sc)","O-Swing%":"FG_O-Swing%","O-Swing% (pi)":"FG_O-Swing% (pi)","O-Swing% (sc)":"FG_O-Swing% (sc)","OBP":"FG_OBP","OBP+":"FG_OBP+","Off":"FG_Off","Oppo%":"FG_Oppo%","Oppo%+":"FG_Oppo%+","OPS":"FG_OPS","PA":"FG_PA","Pace":"FG_Pace","Pace (pi)":"FG_Pace (pi)","PH":"FG_PH","phLI":"FG_phLI","Pitches":"FG_Pitches","playerid":"FG_playerid","pLI":"FG_pLI","PO%":"FG_PO%","Pos":"FG_Pos","Pull%":"FG_Pull%","Pull%+":"FG_Pull%+","R":"FG_R","RAR":"FG_RAR","RBI":"FG_RBI","RE24":"FG_RE24","Rep":"FG_Rep","REW":"FG_REW","SB":"FG_SB","SB-X (pi)":"FG_SB-X (pi)","SB-Z (pi)":"FG_SB-Z (pi)","SB% (pi)":"FG_SB% (pi)","SC-X (sc)":"FG_SC-X (sc)","SC-Z (sc)":"FG_SC-Z (sc)","SC% (sc)":"FG_SC% (sc)","SF":"FG_SF","SF%":"FG_SF%","SFv":"FG_SFv","SH":"FG_SH","SI-X (pi)":"FG_SI-X (pi)","SI-X (sc)":"FG_SI-X (sc)","SI-Z (pi)":"FG_SI-Z (pi)","SI-Z (sc)":"FG_SI-Z (sc)","SI% (pi)":"FG_SI% (pi)","SI% (sc)":"FG_SI% (sc)","SL-X (pi)":"FG_SL-X (pi)","SL-X (sc)":"FG_SL-X (sc)","SL-Z (pi)":"FG_SL-Z (pi)","SL-Z (sc)":"FG_SL-Z (sc)","SL%":"FG_SL%","SL% (pi)":"FG_SL% (pi)","SL% (sc)":"FG_SL% (sc)","SLG":"FG_SLG","SLG+":"FG_SLG+","SLv":"FG_SLv","SO":"FG_SO","Soft%":"FG_Soft%","Soft%+":"FG_Soft%+","Spd":"FG_Spd","Strikes":"FG_Strikes","Swing%":"FG_Swing%","Swing% (pi)":"FG_Swing% (pi)","Swing% (sc)":"FG_Swing% (sc)","SwStr%":"FG_SwStr%","Team":"FG_Team","TTO%":"FG_TTO%","UBR":"FG_UBR","UN% (sc)":"FG_UN% (sc)","vCH (pi)":"FG_vCH (pi)","vCH (sc)":"FG_vCH (sc)","vCS (pi)":"FG_vCS (pi)","vCU (pi)":"FG_vCU (pi)","vCU (sc)":"FG_vCU (sc)","vEP (sc)":"FG_vEP (sc)","vFA (pi)":"FG_vFA (pi)","vFA (sc)":"FG_vFA (sc)","vFC (pi)":"FG_vFC (pi)","vFC (sc)":"FG_vFC (sc)","vFO (sc)":"FG_vFO (sc)","vFS (pi)":"FG_vFS (pi)","vFS (sc)":"FG_vFS (sc)","vFT (sc)":"FG_vFT (sc)","vKC (sc)":"FG_vKC (sc)","vKN (pi)":"FG_vKN (pi)","vKN (sc)":"FG_vKN (sc)","vSB (pi)":"FG_vSB (pi)","vSC (sc)":"FG_vSC (sc)","vSI (pi)":"FG_vSI (pi)","vSI (sc)":"FG_vSI (sc)","vSL (pi)":"FG_vSL (pi)","vSL (sc)":"FG_vSL (sc)","vXX (pi)":"FG_vXX (pi)","WAR":"FG_WAR","wCB":"FG_wCB","wCB/C":"FG_wCB/C","wCH":"FG_wCH","wCH (pi)":"FG_wCH (pi)","wCH (sc)":"FG_wCH (sc)","wCH/C":"FG_wCH/C","wCH/C (pi)":"FG_wCH/C (pi)","wCH/C (sc)":"FG_wCH/C (sc)","wCS (pi)":"FG_wCS (pi)","wCS/C (pi)":"FG_wCS/C (pi)","wCT":"FG_wCT","wCT/C":"FG_wCT/C","wCU (pi)":"FG_wCU (pi)","wCU (sc)":"FG_wCU (sc)","wCU/C (pi)":"FG_wCU/C (pi)","wCU/C (sc)":"FG_wCU/C (sc)","wEP (sc)":"FG_wEP (sc)","wEP/C (sc)":"FG_wEP/C (sc)","wFA (pi)":"FG_wFA (pi)","wFA (sc)":"FG_wFA (sc)","wFA/C (pi)":"FG_wFA/C (pi)","wFA/C (sc)":"FG_wFA/C (sc)","wFB":"FG_wFB","wFB/C":"FG_wFB/C","wFC (pi)":"FG_wFC (pi)","wFC (sc)":"FG_wFC (sc)","wFC/C (pi)":"FG_wFC/C (pi)","wFC/C (sc)":"FG_wFC/C (sc)","wFO (sc)":"FG_wFO (sc)","wFO/C (sc)":"FG_wFO/C (sc)","wFS (pi)":"FG_wFS (pi)","wFS (sc)":"FG_wFS (sc)","wFS/C (pi)":"FG_wFS/C (pi)","wFS/C (sc)":"FG_wFS/C (sc)","wFT (sc)":"FG_wFT (sc)","wFT/C (sc)":"FG_wFT/C (sc)","wGDP":"FG_wGDP","wKC (sc)":"FG_wKC (sc)","wKC/C (sc)":"FG_wKC/C (sc)","wKN":"FG_wKN","wKN (pi)":"FG_wKN (pi)","wKN (sc)":"FG_wKN (sc)","wKN/C":"FG_wKN/C","wKN/C (pi)":"FG_wKN/C (pi)","wKN/C (sc)":"FG_wKN/C (sc)","wOBA":"FG_wOBA","WPA":"FG_WPA","WPA/LI":"FG_WPA/LI","wRAA":"FG_wRAA","wRC":"FG_wRC","wRC+":"FG_wRC+","wSB":"FG_wSB","wSB (pi)":"FG_wSB (pi)","wSB/C (pi)":"FG_wSB/C (pi)","wSC (sc)":"FG_wSC (sc)","wSC/C (sc)":"FG_wSC/C (sc)","wSF":"FG_wSF","wSF/C":"FG_wSF/C","wSI (pi)":"FG_wSI (pi)","wSI (sc)":"FG_wSI (sc)","wSI/C (pi)":"FG_wSI/C (pi)","wSI/C (sc)":"FG_wSI/C (sc)","wSL":"FG_wSL","wSL (pi)":"FG_wSL (pi)","wSL (sc)":"FG_wSL (sc)","wSL/C":"FG_wSL/C","wSL/C (pi)":"FG_wSL/C (pi)","wSL/C (sc)":"FG_wSL/C (sc)","wXX (pi)":"FG_wXX (pi)","wXX/C (pi)":"FG_wXX/C (pi)","xBA":"FG_xBA","xSLG":"FG_xSLG","xwOBA":"FG_xwOBA","XX-X (pi)":"FG_XX-X (pi)","XX-Z (pi)":"FG_XX-Z (pi)","XX%":"FG_XX%","XX% (pi)":"FG_XX% (pi)","Z-Contact%":"FG_Z-Contact%","Z-Contact% (pi)":"FG_Z-Contact% (pi)","Z-Contact% (sc)":"FG_Z-Contact% (sc)","Z-Swing%":"FG_Z-Swing%","Z-Swing% (pi)":"FG_Z-Swing% (pi)","Z-Swing% (sc)":"FG_Z-Swing% (sc)","Zone%":"FG_Zone%","Zone% (pi)":"FG_Zone% (pi)","Zone% (sc)":"FG_Zone% (sc)","-WPA":"FG_-WPA","+WPA":"FG_+WPA","Season":"FG_Season","PlayerTeamSeason":"FG_PlayerTeamSeason"})


In [95]:
#rename FG columns
fg = fg.rename(columns={"FG_PlayerTeamSeason":"PlayerTeamSeason"})


In [96]:
#drop all columns with NAN
fg = fg.dropna(axis=1, how='any')

In [97]:
#remove FG_Age Rng
fg = fg.drop(['FG_Age Rng'], axis=1)


In [98]:
#grab only columns we want to merge with SH
fg_cols = fg[["FG_AVG","FG_BABIP","FG_Bat","FG_BB/K","FG_BB%","FG_BsR","FG_ISO","FG_K%","FG_L-WAR","FG_Def","FG_Events","FG_playerid","FG_SLG","FG_Spd","FG_WAR","FG_wOBA","FG_wRAA","FG_wRC","FG_wSB","FG_OBP","FG_SLG","FG_OPS","PlayerTeamSeason"]].copy()


In [99]:
#prep stathead data

In [100]:
sh['PlayerTeamSeason'] = sh['Player'].str.replace(" ", "_") + '_' + sh['Team'] + '_' + sh['Season'].map(str)

In [101]:
#merge stathead data
df_comb = pd.merge(sh, fg_cols, how="inner", on=['PlayerTeamSeason'])

In [102]:
#missing values in SH BA, OBP, SLG, OPS - Use FG
df_comb = df_comb.drop(["BA","OBP","SLG","OPS"], axis=1)

In [103]:
#missing values in OPS+ - no FG equiv, change to -100.0
df_comb["OPS+"] = df_comb["OPS+"].fillna(-100.0)

In [104]:
#drop dupe WAR.1
df_comb = df_comb.drop(["WAR.1"], axis=1)

In [105]:
#group by playerid and then remove players with only 1 season
df_comb = df_comb.groupby("FG_playerid", group_keys=False).filter(lambda x: x.shape[0] > 1)


In [106]:
#get cols with na values
null_count = df_comb.isnull().sum()


In [107]:
#get list of complete cols
complete_cols = list(df_comb.columns[null_count ==0])


In [108]:
#function to add next season WAR to record
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player

#add Next_WAR to record
df_comb_prev_next = df_comb.groupby("FG_playerid", group_keys=False).apply(next_season)


In [109]:
#function to add previous season WAR to record
def prev_season(player):
    player = player.sort_values("Season")
    player["Prev_WAR"] = player["WAR"].shift(1)
    return player

#Add Prev_WAR to record
df_comb_prev_next = df_comb_prev_next.groupby("FG_playerid", group_keys=False).apply(prev_season)

In [110]:
#check a player
df_comb_prev_next[["Player","Season","Prev_WAR","WAR","Next_WAR"]][df_comb_prev_next["Player"] == "Barry Bonds"]


,Player,Season,Prev_WAR,WAR,Next_WAR
2049,Barry Bonds,1986,NaN,3.5,5.8
545,Barry Bonds,1987,3.5,5.8,6.3
403,Barry Bonds,1988,5.8,6.3,8.0
80,Barry Bonds,1989,6.3,8.0,9.7
14,Barry Bonds,1990,8.0,9.7,8.0
83,Barry Bonds,1991,9.7,8.0,9.0
31,Barry Bonds,1992,8.0,9.0,9.9
10,Barry Bonds,1993,9.0,9.9,6.2
405,Barry Bonds,1994,9.9,6.2,7.5
136,Barry Bonds,1995,6.2,7.5,9.7


In [111]:
#remove null colums
df_comb_prev_next = df_comb_prev_next[complete_cols + ["Prev_WAR"] + ["Next_WAR"]].copy()

In [112]:
#ingest positional w innings csv
pos_inn = pd.read_csv('pos/pos_inn.csv')

In [113]:
#Add PlayerTeamSeason for individual seasons
pos_inn['PlayerTeamSeason'] = pos_inn['Name'].str.replace(" ", "_") + '_' + pos_inn['Team'] + '_' + pos_inn['Season'].map(str)
pos_inn


,Season,Name,Team,Pos,Inn,playerid,PlayerTeamSeason
0,2010,Yadier Molina,STL,C,1138.0,7007,Yadier_Molina_STL_2010
1,2008,Brian McCann,ATL,C,1143.1,4810,Brian_McCann_ATL_2008
2,2008,Russell Martin,LAD,C,1238.0,4616,Russell_Martin_LAD_2008
3,2008,Jose Molina,NYY,C,737.0,25,Jose_Molina_NYY_2008
4,2013,Jonathan Lucroy,MIL,C,1074.0,7870,Jonathan_Lucroy_MIL_2013
...,...,...,...,...,...,...,...
77802,2022,Ken Waldichuk,OAK,P,34.2,27681,Ken_Waldichuk_OAK_2022
77803,2022,Louie Varland,MIN,P,26.0,27691,Louie_Varland_MIN_2022
77804,2022,Bryce Elder,ATL,P,54.0,27779,Bryce_Elder_ATL_2022
77805,2022,Chase Silseth,LAA,P,28.2,30074,Chase_Silseth_LAA_2022


In [114]:
#Isolate P 
pos_inn_p = pos_inn[pos_inn["Pos"]=="P"]
#Isolate Not P
pos_inn_pos = pos_inn[pos_inn["Pos"]!="P"]

In [115]:
#get of
pos_inn_pos_of = pos_inn_pos[(pos_inn_pos["Pos"] == "LF") | (pos_inn_pos["Pos"] == "RF") | (pos_inn_pos["Pos"] == "CF")]


In [116]:
#sum of
pos_inn_pos_of_gp = pos_inn_pos_of.groupby(by="PlayerTeamSeason")["Inn"].sum()

In [117]:
#get rid of duplicates
pos_inn_pos_of_filt = pos_inn_pos_of.drop_duplicates(subset='PlayerTeamSeason', keep="last")


In [118]:
#merge OF
pos_inn_pos_of_all = pd.merge(pos_inn_pos_of_filt, pos_inn_pos_of_gp, how="inner", on=['PlayerTeamSeason'])


In [119]:
#drop dupe Inn
pos_inn_pos_of_all = pos_inn_pos_of_all.drop(['Inn_x'], axis=1)
#rename Inn column
pos_inn_pos_of_all = pos_inn_pos_of_all.rename(columns={"Inn_y":"Inn"})

In [120]:
#add OF col
pos_inn_pos_of_all["Pos_Cat"] = "OF"

In [121]:
#drop dupe Inn
pos_inn_pos_of_all = pos_inn_pos_of_all.drop(['Pos'], axis=1)

In [122]:
#remove of from pos
pos_inn_pos_no_of = pos_inn_pos[(pos_inn_pos["Pos"]!="LF") & (pos_inn_pos["Pos"]!="RF") & (pos_inn_pos["Pos"]!="CF")]


In [123]:
#rename pos col
pos_inn_pos_no_of = pos_inn_pos_no_of.rename(columns={"Pos":"Pos_Cat"})


In [124]:
#concat dfs
pos_inn_final = pd.concat([pos_inn_pos_no_of,pos_inn_pos_of_all])



In [125]:
#rename FG columns
pos_inn_final = pos_inn_final.rename(columns={"playerid":"FG_playerid"})


In [126]:
#group by PlayerTeamSeason and keep highest Inn total for duplicates
pos_inn_final = pos_inn_final.groupby('PlayerTeamSeason', group_keys=False).apply(lambda x: x.loc[x.Inn.idxmax()])


In [127]:
#ingest positional w innings csv
pos_inn_dh = pd.read_csv('pos/pos_inn_dh.csv')

In [128]:
#add PlayerTeamSeason
pos_inn_dh['PlayerTeamSeason'] = pos_inn_dh['Name'].str.replace(" ", "_") + '_' + pos_inn_dh['Team'] + '_' + pos_inn_dh['Season'].map(str)


In [129]:
#create col to merge
pos_inn_dh['idseason'] = pos_inn_dh['playerid'].map(str) + '_' + pos_inn_dh['Season'].map(str)

In [130]:
pos_inn_dh = pos_inn_dh.set_index('PlayerTeamSeason')

In [131]:
pos_inn_dh_merge = pos_inn_dh[["idseason","Pos_Cat"]]

In [132]:
#need to add positional data to individual season date
#df_comb_prev_next - merged SH and FG date with previous and next season's WAR values (based on SH WAR)*
#df3 - merged data, no next/prev cols*


In [133]:
#create col to merge
pos_inn_final['idseason'] = pos_inn_final['FG_playerid'].map(str) + '_' + pos_inn_final['Season'].map(str)

In [134]:
#col to dfs
df_comb_prev_next['idseason'] = df_comb_prev_next['FG_playerid'].map(str) + '_' + df_comb_prev_next['Season'].map(str)


In [135]:
#df for merge
pos_inn_final_merge = pos_inn_final[["idseason","Pos_Cat"]]

In [136]:
#add pos and dh
pos_inn_final_merge = pd.concat([pos_inn_final_merge, pos_inn_dh_merge])


In [137]:
#merge position
df_comb_prev_next_pos = pd.merge(df_comb_prev_next, pos_inn_final_merge, how="inner", on=['idseason'])
#df3 = pd.merge(df3, pos_inn_final_merge, how="inner", on=['idseason'])


In [138]:
#INVENTORY
#df_comb_prev_next_pos - merged SH and FG date with previous and next season's WAR values (based on SH WAR)*
#df3 - merged data, no next/prev cols*
#df_gp_avg_comb - merges df_obj and df_avg_gp by ID z

# *merge with position
# zmerge with peak season data

#at this point, need DH data to merge in

In [139]:
#get peak OF records
df_comb_prev_next_pos_OF_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 27) & (df_comb_prev_next_pos['Pos_Cat'] == 'OF')]
#get peak SS records
df_comb_prev_next_pos_SS_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 27) & (df_comb_prev_next_pos['Pos_Cat'] == 'SS')]
#get only peak 1B records
df_comb_prev_next_pos_1B_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 29) & (df_comb_prev_next_pos['Pos_Cat'] == '1B')]
#get only peak 2b records
df_comb_prev_next_pos_2B_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 29) & (df_comb_prev_next_pos['Pos_Cat'] == '2B')]
#get only peak 3b records
df_comb_prev_next_pos_3B_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 28) & (df_comb_prev_next_pos['Pos_Cat'] == '3B')]
#get only peak C records
df_comb_prev_next_pos_C_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 30) & (df_comb_prev_next_pos['Pos_Cat'] == 'C')]


In [140]:
#get only peak DH records
df_comb_prev_next_pos_DH_peak = df_comb_prev_next_pos[(df_comb_prev_next_pos['Age'] == 35) & (df_comb_prev_next_pos['Pos_Cat'] == 'DH')]


In [141]:
#concat peaks
df_comb_prev_next_pos_peaks = pd.concat([df_comb_prev_next_pos_OF_peak, df_comb_prev_next_pos_SS_peak, df_comb_prev_next_pos_3B_peak, df_comb_prev_next_pos_1B_peak, df_comb_prev_next_pos_2B_peak, df_comb_prev_next_pos_C_peak, df_comb_prev_next_pos_DH_peak])


In [142]:
#rename PEAK columns
df_comb_prev_next_pos_peaks = df_comb_prev_next_pos_peaks.rename(columns={"Rk":"Peak_Rk","WAR":"Peak_WAR","Season":"Peak_Season","Age":"Peak_Age","Team":"Peak_Team","Lg":"Peak_Lg","G":"Peak_G","PA":"Peak_PA","AB":"Peak_AB","R":"Peak_R","H":"Peak_H","1B":"Peak_1B","2B":"Peak_2B","3B":"Peak_3B","HR":"Peak_HR","RBI":"Peak_RBI","SB":"Peak_SB","CS":"Peak_CS","BB":"Peak_BB","SO":"Peak_SO","OPS+":"Peak_OPS+","TB":"Peak_TB","GIDP":"Peak_GIDP","HBP":"Peak_HBP","SH":"Peak_SH","SF":"Peak_SF","IBB":"Peak_IBB","WAA":"Peak_WAA","oWAR":"Peak_oWAR","dWAR":"Peak_dWAR","Rbat":"Peak_Rbat","Rdp":"Peak_Rdp","Rbaser":"Peak_Rbaser","Rbaser+Rdp":"Peak_Rbaser+Rdp","Rfield":"Peak_Rfield","Pos":"Peak_Pos","PlayerTeamSeason":"Peak_PlayerTeamSeason","FG_AVG":"Peak_FG_AVG","FG_BABIP":"Peak_FG_BABIP","FG_Bat":"Peak_FG_Bat","FG_BB/K":"Peak_FG_BB/K","FG_BB%":"Peak_FG_BB%","FG_BsR":"Peak_FG_BsR","FG_ISO":"Peak_FG_ISO","FG_K%":"Peak_FG_K%","FG_L-WAR":"Peak_FG_L-WAR","FG_Def":"Peak_FG_Def","FG_Events":"Peak_FG_Events","FG_SLG":"Peak_FG_SLG","FG_SLG":"Peak_FG_SLG","FG_Spd":"Peak_FG_Spd","FG_WAR":"Peak_FG_WAR","FG_wOBA":"Peak_FG_wOBA","FG_wRAA":"Peak_FG_wRAA","FG_wRC":"Peak_FG_wRC","FG_wSB":"Peak_FG_wSB","FG_OBP":"Peak_FG_OBP","FG_SLG":"Peak_FG_SLG","FG_SLG":"Peak_FG_SLG","FG_OPS":"Peak_FG_OPS","Prev_WAR":"Peak_Prev_WAR","Next_WAR":"Peak_Next_WAR","idseason":"Peak_idseason","Pos_Cat":"Peak_Pos_Cat","Inn":"Peak_Inn"})


In [143]:
#remove Player
df_comb_prev_next_pos_peaks = df_comb_prev_next_pos_peaks.drop(['Player'], axis=1)

In [144]:
#merge ss and peak
ss_pos_df = pd.merge(df_comb_prev_next, df_comb_prev_next_pos_peaks, how="inner", on=['FG_playerid'])

In [145]:
#Check
ss_pos_df

,Rk,Player,WAR,Season,Age,Team,Lg,G,PA,AB,...,Peak_FG_wRC,Peak_FG_wSB,Peak_FG_OBP,Peak_FG_SLG,Peak_FG_SLG,Peak_FG_OPS,Peak_Prev_WAR,Peak_Next_WAR,Peak_idseason,Peak_Pos_Cat
0,14374,Garret Anderson,0.1,1994,22,CAL,AL,5,13,13,...,88,-1.2,0.336,0.469,0.469,0.806,0.3,1.9,2_1999,OF
1,3131,Garret Anderson,3.0,1995,23,CAL,AL,106,400,374,...,88,-1.2,0.336,0.469,0.469,0.806,0.3,1.9,2_1999,OF
2,9220,Garret Anderson,0.8,1996,24,CAL,AL,150,642,607,...,88,-1.2,0.336,0.469,0.469,0.806,0.3,1.9,2_1999,OF
3,2690,Garret Anderson,3.3,1997,25,ANA,AL,154,662,624,...,88,-1.2,0.336,0.469,0.469,0.806,0.3,1.9,2_1999,OF
4,12775,Garret Anderson,0.3,1998,26,ANA,AL,156,658,622,...,88,-1.2,0.336,0.469,0.469,0.806,0.3,1.9,2_1999,OF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15712,20539,Paul Zuvella,-0.3,1985,26,ATL,NL,81,210,190,...,-1,0.0,0.170,0.104,0.104,0.274,-0.3,-0.5,1014465_1986,SS
15713,24410,Paul Zuvella,-0.8,1986,27,NYY,AL,21,57,48,...,-1,0.0,0.170,0.104,0.104,0.274,-0.3,-0.5,1014465_1986,SS
15714,22817,Paul Zuvella,-0.5,1987,28,NYY,AL,14,36,34,...,-1,0.0,0.170,0.104,0.104,0.274,-0.3,-0.5,1014465_1986,SS
15715,23947,Paul Zuvella,-0.7,1988,29,CLE,AL,51,146,130,...,-1,0.0,0.170,0.104,0.104,0.274,-0.3,-0.5,1014465_1986,SS


In [146]:
#exceed Prev_WAR in Peak? 0 if no, 1 if yes
#exceed P_WAR in Next_Peak?  0 if no, 1 if yes
def prev_war(x):
    if x["Peak_Prev_WAR"] >= x["Peak_WAR"]:
        return 0
    if x["Peak_Prev_WAR"] < x["Peak_WAR"]:
        return 1
    return ''

ss_pos_df['Exceed_Prev_WAR']=ss_pos_df.apply(lambda x: prev_war(x), axis=1)

def p_war(x):
    if x["Peak_WAR"] >= x["Peak_Next_WAR"]:
        return 0
    if x["Peak_WAR"] < x["Peak_Next_WAR"]:
        return 1
    return ''

ss_pos_df['Exceed_Peak_WAR']=ss_pos_df.apply(lambda x: p_war(x), axis=1)


In [147]:
#check one
ss_pos_df[["FG_playerid","Season","Peak_Season","Age","Peak_Age","Player","Prev_WAR","WAR","Next_WAR","Peak_Prev_WAR","Peak_WAR","Peak_Next_WAR","Exceed_Prev_WAR","Exceed_Peak_WAR"]][ss_pos_df["Player"] == "J.J. Hardy"]


,FG_playerid,Season,Peak_Season,Age,Peak_Age,Player,Prev_WAR,WAR,Next_WAR,Peak_Prev_WAR,Peak_WAR,Peak_Next_WAR,Exceed_Prev_WAR,Exceed_Peak_WAR
6204,3797,2005,2010,22,27,J.J. Hardy,NaN,1.8,0.8,0.5,1.3,4.1,1,1
6205,3797,2006,2010,23,27,J.J. Hardy,1.8,0.8,3.2,0.5,1.3,4.1,1,1
6206,3797,2007,2010,24,27,J.J. Hardy,0.8,3.2,4.8,0.5,1.3,4.1,1,1
6207,3797,2008,2010,25,27,J.J. Hardy,3.2,4.8,0.5,0.5,1.3,4.1,1,1
6208,3797,2009,2010,26,27,J.J. Hardy,4.8,0.5,1.3,0.5,1.3,4.1,1,1
6209,3797,2010,2010,27,27,J.J. Hardy,0.5,1.3,4.1,0.5,1.3,4.1,1,1
6210,3797,2011,2010,28,27,J.J. Hardy,1.3,4.1,3.3,0.5,1.3,4.1,1,1
6211,3797,2012,2010,29,27,J.J. Hardy,4.1,3.3,3.6,0.5,1.3,4.1,1,1
6212,3797,2013,2010,30,27,J.J. Hardy,3.3,3.6,3.1,0.5,1.3,4.1,1,1
6213,3797,2014,2010,31,27,J.J. Hardy,3.6,3.1,-0.3,0.5,1.3,4.1,1,1


In [148]:
#ss_pos_df[(ss_pos_df['Peak_Season'] == 2022) & (ss_pos_df['Season'] == 2022)]

#check one
ss_pos_df[["FG_playerid","Season","Peak_Season","Age","Peak_Age","Player","Prev_WAR","WAR","Next_WAR","Peak_Prev_WAR","Peak_WAR","Peak_Next_WAR","Exceed_Prev_WAR","Exceed_Peak_WAR"]][ss_pos_df["Player"] == "Carlos Correa"]


,FG_playerid,Season,Peak_Season,Age,Peak_Age,Player,Prev_WAR,WAR,Next_WAR,Peak_Prev_WAR,Peak_WAR,Peak_Next_WAR,Exceed_Prev_WAR,Exceed_Peak_WAR
9414,14162,2015,2022,20,27,Carlos Correa,NaN,4.8,7.0,7.2,5.4,NaN,0,
9415,14162,2016,2022,21,27,Carlos Correa,4.8,7.0,6.7,7.2,5.4,NaN,0,
9416,14162,2017,2022,22,27,Carlos Correa,7.0,6.7,3.1,7.2,5.4,NaN,0,
9417,14162,2018,2022,23,27,Carlos Correa,6.7,3.1,3.7,7.2,5.4,NaN,0,
9418,14162,2019,2022,24,27,Carlos Correa,3.1,3.7,1.7,7.2,5.4,NaN,0,
9419,14162,2020,2022,25,27,Carlos Correa,3.7,1.7,7.2,7.2,5.4,NaN,0,
9420,14162,2021,2022,26,27,Carlos Correa,1.7,7.2,5.4,7.2,5.4,NaN,0,
9421,14162,2022,2022,27,27,Carlos Correa,7.2,5.4,NaN,7.2,5.4,NaN,0,


In [149]:
#clean up objects
#object: ['Player', 'Team', 'Lg', 'PlayerTeamSeason', 'Peak_Team', 'Peak_Lg', 'Peak_PlayerTeamSeason', 'Exceed_Prev_WAR', 'Exceed_Peak_WAR']}

#drop player
ss_pos_df = ss_pos_df.drop(['PlayerTeamSeason','Peak_PlayerTeamSeason'], axis=1)

#AL = 0, NL = 1
ss_pos_df['Lg'] = ss_pos_df['Lg'].map({'AL':0, 'NL':1})
ss_pos_df['Peak_Lg'] = ss_pos_df['Peak_Lg'].map({'AL':0, 'NL':1})

#Team
ss_pos_df['Team'] = ss_pos_df['Team'].map({'ARI':0,'ATL':1,'BAL':2,'BOS':3,'CAL':4,'ANA':4,'LAA':4,'CHC':5,'CHW':6,'CIN':7,'CLE':8,'COL':9,'DET':10,'FLA':11,'MIA':11,'HOU':12,'KCR':13,'LAD':14,'MIL':15,'MIN':16,'NYM':17,'NYY':18,'OAK':19,'PHI':20,'PIT':21,'SDP':22,'SEA':23,'SFG':24,'STL':25,'TBD':26,'TBR':26,'TEX':27,'TOR':28,'WSN':29,'MON':29})
ss_pos_df['Peak_Team'] = ss_pos_df['Peak_Team'].map({'ARI':0,'ATL':1,'BAL':2,'BOS':3,'CAL':4,'ANA':4,'LAA':4,'CHC':5,'CHW':6,'CIN':7,'CLE':8,'COL':9,'DET':10,'FLA':11,'MIA':11,'HOU':12,'KCR':13,'LAD':14,'MIL':15,'MIN':16,'NYM':17,'NYY':18,'OAK':19,'PHI':20,'PIT':21,'SDP':22,'SEA':23,'SFG':24,'STL':25,'TBD':26,'TBR':26,'TEX':27,'TOR':28,'WSN':29,'MON':29})


In [150]:
#drop Peak_Prev_WAR and Exceed_Prev_WAR
#ss_pos_df = ss_pos_df.drop(['Peak_Prev_WAR','Exceed_Prev_WAR'], axis=1)

In [151]:
#check value counts
ss_pos_df['Exceed_Peak_WAR'].value_counts()

0    7912
1    6328
     1477
Name: Exceed_Peak_WAR, dtype: int64

In [152]:
#drop Pos and Peak_Pos
ss_pos_df = ss_pos_df.drop(['Pos','Peak_Pos','Peak_idseason','idseason'], axis=1)



In [153]:
#C = 0, 1B = 1, 2B = 2, 3B = 3, SS = 4, OF = 5, DH = 6
ss_pos_df['Peak_Pos_Cat'] = ss_pos_df['Peak_Pos_Cat'].map({'C':0, '1B':1, '2B':2, '3B':3, 'SS':4, 'OF':5, 'DH':6})

In [154]:
#fix %s
ss_pos_df['FG_BB%'] = ss_pos_df['FG_BB%'].map(lambda x: x.rstrip('%')).astype(float)/100
ss_pos_df['FG_K%'] = ss_pos_df['FG_K%'].map(lambda x: x.rstrip('%')).astype(float)/100
ss_pos_df['Peak_FG_BB%'] = ss_pos_df['Peak_FG_BB%'].map(lambda x: x.rstrip('%')).astype(float)/100
ss_pos_df['Peak_FG_K%'] = ss_pos_df['Peak_FG_K%'].map(lambda x: x.rstrip('%')).astype(float)/100


In [155]:
#all teams 1980-2022
sh_td = pd.read_csv('team_data.csv')
sh_td.head(10)

,Season,Team,Team_GP,Team_WL
0,2002,LAA,162,0.611
1,2004,LAA,162,0.568
2,1998,LAA,162,0.525
3,1997,LAA,162,0.519
4,2000,LAA,162,0.506
5,2003,LAA,162,0.475
6,2001,LAA,162,0.463
7,1999,LAA,162,0.432
8,1999,ARI,162,0.617
9,2002,ARI,162,0.605


In [156]:
sh_td['Team'] = sh_td['Team'].map({'ARI':0,'ATL':1,'BAL':2,'BOS':3,'CAL':4,'ANA':4,'LAA':4,'CHC':5,'CHW':6,'CIN':7,'CLE':8,'COL':9,'DET':10,'FLA':11,'MIA':11,'HOU':12,'KCR':13,'LAD':14,'MIL':15,'MIN':16,'NYM':17,'NYY':18,'OAK':19,'PHI':20,'PIT':21,'SDP':22,'SEA':23,'SFG':24,'STL':25,'TBD':26,'TBR':26,'TEX':27,'TOR':28,'WSN':29,'MON':29})


In [157]:
sh_td

,Season,Team,Team_GP,Team_WL
0,2002,4,162,0.611
1,2004,4,162,0.568
2,1998,4,162,0.525
3,1997,4,162,0.519
4,2000,4,162,0.506
...,...,...,...,...
1223,2021,29,162,0.401
1224,2009,29,162,0.364
1225,2008,29,161,0.366
1226,2022,29,162,0.340


In [158]:
ss_pos_df['SeasonTeam'] = ss_pos_df['Season'].map(str) + '_' + ss_pos_df['Team'].map(str)
sh_td['SeasonTeam'] = sh_td['Season'].map(str) + '_' + sh_td['Team'].map(str)

In [159]:
ss_pos_df['SeasonTeam'] = ss_pos_df['Season'].map(str) + '_' + ss_pos_df['Team'].map(str)

In [160]:
team_data = sh_td.copy()
team_data = team_data.drop(['Season','Team'], axis=1)
team_data


,Team_GP,Team_WL,SeasonTeam
0,162,0.611,2002_4
1,162,0.568,2004_4
2,162,0.525,1998_4
3,162,0.519,1997_4
4,162,0.506,2000_4
...,...,...,...
1223,162,0.401,2021_29
1224,162,0.364,2009_29
1225,161,0.366,2008_29
1226,162,0.340,2022_29


In [161]:
#merge team data
ss_pos_df = pd.merge(ss_pos_df, team_data, how="inner", on=['SeasonTeam'])
ss_pos_df = ss_pos_df.drop(['SeasonTeam'], axis=1)
ss_pos_df["Games_played"] = ss_pos_df["G"]/ss_pos_df["Team_GP"]
ss_pos_df = ss_pos_df.drop(['Team_GP'], axis=1)
ss_pos_df

,Rk,Player,WAR,Season,Age,Team,Lg,G,PA,AB,...,Peak_FG_SLG,Peak_FG_SLG,Peak_FG_OPS,Peak_Prev_WAR,Peak_Next_WAR,Peak_Pos_Cat,Exceed_Prev_WAR,Exceed_Peak_WAR,Team_WL,Games_played
0,14374,Garret Anderson,0.1,1994,22,4,0,5,13,13,...,0.469,0.469,0.806,0.3,1.9,5,1,0,0.409,0.043478
1,23359,Jorge Fabregas,-0.6,1994,24,4,0,43,135,127,...,0.373,0.373,0.693,-0.3,-0.4,0,1,0,0.409,0.373913
2,3805,Tim Salmon,2.6,1994,25,4,0,100,435,373,...,0.501,0.501,0.887,6.6,5.0,5,0,1,0.409,0.869565
3,3805,Tim Salmon,2.6,1994,25,4,0,100,435,373,...,0.323,0.323,0.628,2.8,0.7,6,0,1,0.409,0.869565
4,22287,Damion Easley,-0.4,1994,24,4,0,88,355,316,...,0.434,0.434,0.779,5.6,4.0,2,0,1,0.409,0.765217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15710,22050,Darrell Miller,-0.4,1984,26,4,0,17,45,41,...,0.307,0.307,0.599,0.4,NaN,0,0,,0.500,0.104938
15711,14752,Jerry Narron,0.1,1984,28,4,0,69,162,150,...,0.305,0.305,0.598,0.1,-0.2,0,0,0,0.500,0.425926
15712,5281,Gary Pettis,2.0,1984,26,4,0,140,466,397,...,0.323,0.323,0.670,2.0,5.1,5,1,1,0.500,0.864198
15713,17394,Dick Schofield,-0.1,1984,21,4,0,140,452,400,...,0.297,0.297,0.660,2.5,0.8,4,0,0,0.500,0.864198


In [162]:
#export out
from pathlib import Path  
filepath = Path('ss_pos_df_clean.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
ss_pos_df.to_csv(filepath,index=False)  